In [ ]:

# 🚖 UBER FARE PREDICTION
# Step-by-step machine learning project (beginner-friendly)

# -------------------------------------
# 1️⃣ Import all required libraries
# -------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# -------------------------------------
# 2️⃣ Load and inspect the dataset
# -------------------------------------
df = pd.read_csv("uber.csv")
print("Initial shape:", df.shape)
print(df.info())
print(df.isnull().sum())

# -------------------------------------
# 3️⃣ Handle missing values
# -------------------------------------
df.dropna(inplace=True)
print("\nAfter removing missing values:", df.shape)

# -------------------------------------
# 4️⃣ Feature Engineering — Calculate Distance using Haversine formula
# -------------------------------------
def haversine(lon1, lon2, lat1, lat2):
    lon1, lon2, lat1, lat2 = map(np.radians, [lon1, lon2, lat1, lat2])
    diff_lon = lon2 - lon1
    diff_lat = lat2 - lat1
    km = 2 * 6371 * np.arcsin(np.sqrt(np.sin(diff_lat/2.0)**2 +
                                      np.cos(lat1) * np.cos(lat2) * np.sin(diff_lon/2.0)**2))
    return km

df['Distance'] = haversine(df['pickup_longitude'], df['dropoff_longitude'],
                           df['pickup_latitude'], df['dropoff_latitude']).round(2)

# -------------------------------------
# 5️⃣ Remove Outliers and invalid values
# -------------------------------------
df = df[df['Distance'] > 0]
df = df[df['Distance'] <= 60]
df = df[df['fare_amount'] > 0]
df = df[df['fare_amount'] <= 100]

print("\nAfter cleaning data:", df.shape)

# -------------------------------------
# 6️⃣ Basic Visualization
# -------------------------------------
plt.figure(figsize=(6,4))
plt.scatter(df['Distance'], df['fare_amount'], alpha=0.3)
plt.xlabel("Distance (km)")
plt.ylabel("Fare Amount ($)")
plt.title("Distance vs Fare Amount")
plt.show()

sns.histplot(df['fare_amount'], bins=30, color='teal')
plt.title("Fare Amount Distribution")
plt.show()

# -------------------------------------
# 7️⃣ Prepare data for training
# -------------------------------------
X = df[['Distance']]
y = df['fare_amount']

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# -------------------------------------
# 8️⃣ Linear Regression Model
# -------------------------------------
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

lr_mae = mean_absolute_error(y_test, y_pred_lr)
lr_rmse = np.sqrt(mean_squared_error(y_test, y_pred_lr))
lr_r2 = r2_score(y_test, y_pred_lr)

print("\n📊 Linear Regression Results:")
print("MAE :", round(lr_mae, 3))
print("RMSE:", round(lr_rmse, 3))
print("R²  :", round(lr_r2, 3))

# -------------------------------------
# 9️⃣ Random Forest Regressor Model
# -------------------------------------
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train.ravel())
y_pred_rf = rf_model.predict(X_test)

rf_mae = mean_absolute_error(y_test, y_pred_rf)
rf_rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
rf_r2 = r2_score(y_test, y_pred_rf)

print("\n🌲 Random Forest Results:")
print("MAE :", round(rf_mae, 3))
print("RMSE:", round(rf_rmse, 3))
print("R²  :", round(rf_r2, 3))

# -------------------------------------
# 🔟 Model Comparison
# -------------------------------------
results = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest'],
    'MAE': [lr_mae, rf_mae],
    'RMSE': [lr_rmse, rf_rmse],
    'R² Score': [lr_r2, rf_r2]
})

print("\n✅ Model Comparison:")
print(results)
